# Export the Track Gallery - the others do not put the Hight/Width info in the HTML

In [8]:
import re

def extract_images_from_lightroom(file_path):
    """Extract image data from Lightroom HTML."""
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    # Locate the LR.images block
    start_index = content.find("LR.images = [")
    if start_index == -1:
        raise ValueError("Could not find 'LR.images = [' in the file.")

    start_index += len("LR.images = [")  # Move past 'LR.images = ['
    end_index = content.find("]", start_index)
    if end_index == -1:
        raise ValueError("Could not find the closing ']' for LR.images.")

    # Extract the block as raw text
    raw_data = content[start_index:end_index + 1]

    # Split the raw data into lines or individual objects
    raw_items = re.split(r"},\s*{", raw_data.strip()[1:-1])  # Split by objects
    images = []

    for item in raw_items:
        # Ensure each item is properly wrapped with braces
        item = "{" + item.strip() + "}"
        item = item.replace("&quot;", '"')  # Replace HTML entities with quotes

        # Extract key-value pairs manually
        fields = re.findall(r'"(\w+)":\s*"([^"]*?)"', item)
        image_data = {}

        for key, value in fields:
            # Parse numeric fields as integers, leave others as strings
            if key in {"largeWidth", "largeHeight"}:
                try:
                    image_data[key] = int(value)
                except ValueError:
                    image_data[key] = 0  # Fallback for invalid integers
            else:
                image_data[key] = value.strip()

        # Ensure required keys exist
        if "largeWidth" in image_data and "largeHeight" in image_data:
            images.append(image_data)
        else:
            print(f"Skipping image due to missing dimensions: {item}")

    return images

In [9]:
def generate_header():
    """Generate the HTML header."""
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <title>Pascal Reusch Photography</title>
        <!-- Include Bootstrap -->
        <link href="css/bootstrap-4.4.1.css" rel="stylesheet">
        <link href="css/lightbox.min.css" rel="stylesheet">
    </head>
    <body>
    """


In [10]:
def generate_navbar():
    """Generate the complete navbar."""
    return """
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <a class="navbar-brand" href="#"></a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
            <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navbarSupportedContent">
            <ul class="navbar-nav mr-auto">
                <li class="nav-item">
                    <a class="nav-link" href="index.html">Home</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" href="portraits-manual.html">Portraits</a>
                </li>
                <li class="nav-item active">
                    <a class="nav-link" href="street.html">Street<span class="sr-only">(current)</span></a>
                </li>
            </ul>
        </div>
    </nav>
    """


In [11]:
def generate_jumbotron():
    """Generate the jumbotron section."""
    return """
    <section>
        <div class="jumbotron text-center mt-2 py-2">
            <div class="container">
                <div class="row">
                    <div class="col-12">
                        <h1>Street Photography</h1>
                    </div>
                </div>
            </div>
        </div>
    </section>
    """


def generate_image_section(images, lightroom_folder):
    """Generate the image gallery section."""
    html_content = '<section><div class="container">'
    portrait_row = []
    landscape_row = []

    for image in images:

        print("image looks like this:")
        print(image)
        # Determine orientation of the image
        orientation = "portrait" if int(image["largeHeight"]) > int(image["largeWidth"]) else "landscape"

        # Create the basic image item without column definitions
        html_item = f"""
            <a href="{lightroom_folder}/images/large/{image['exportFilename']}.jpg" data-lightbox="street-gallery" data-title="{image['title']}">
            <img class="img-fluid" src="{lightroom_folder}/images/large/{image['exportFilename']}.jpg" alt="{image['title']}"> </a>
            <h5>{image['title']}</h5>
        """

        if orientation == "portrait":
            portrait_row.append(html_item)
            if len(portrait_row) == 3:  # Once 3 portraits are collected, add the row
                html_content += '<div class="row mt-3">' + ''.join(
                    f'<div class="col-md-4 col-12 text-center">{img}</div>' for img in portrait_row
                ) + '</div>'
                portrait_row = []
        else:  # Landscape
            landscape_row.append(html_item)
            if len(landscape_row) == 2:  # Once 2 landscapes are collected, add the row
                html_content += '<div class="row mt-3">' + ''.join(
                    f'<div class="col-md-6 col-12 text-center">{img}</div>' for img in landscape_row
                ) + '</div>'
                landscape_row = []

    # Handle any remaining images
    if portrait_row:
        col_size = 12 // len(portrait_row)  # Dynamically adjust column size
        html_content += '<div class="row mt-3">' + ''.join(
            f'<div class="col-md-{col_size} col-12 text-center">{img}</div>' for img in portrait_row
        ) + '</div>'

    if landscape_row:
        col_size = 12 // len(landscape_row)  # Dynamically adjust column size
        html_content += '<div class="row mt-3">' + ''.join(
            f'<div class="col-md-{col_size} col-12 text-center">{img}</div>' for img in landscape_row
        ) + '</div>'

    html_content += '</div></section>'
    return html_content

def generate_quote():
    """Generate the footer."""
    return """
    <hr>
    <section>
      <div class="container">
        <div class="row">
          <div class="col-12 text-center">
            <h4>“To me, photography is an art of observation. It’s about finding something interesting in an ordinary place…I’ve found it has little to do with the things you see and everything to do with the way you see them.”</h4>
			  <h4>Elliott Erwitt</h4>
          </div>
        </div>
      </div>
    </section>
    """

def generate_footer():
    """Generate the footer."""
    return """
    <footer class="text-center">
        <div class="container">
            <div class="row">
                <div class="col-12">
                    <p>Copyright © Pascal Reusch Photography. All rights reserved.</p>
                </div>
            </div>
        </div>
    </footer>
    <!-- Include Bootstrap JavaScript -->
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="js/popper.min.js"></script>
    <script src="js/bootstrap-4.4.1.js"></script>
    <script src="js/lightbox.min.js"></script>
    </body>
    </html>
    """


def generate_full_html(images, lightroom_folder):
    """Combine all sections to create the full HTML."""
    return (
        generate_header()
        + generate_navbar()
        + generate_jumbotron()
        + generate_image_section(images, lightroom_folder)
        + generate_quote()
        + generate_footer()
    )

In [12]:

def main():
    # File paths
    lightroom_folder = "LR_Export_Street"
    lightroom_file = lightroom_folder+"/index.html"
    output_html_file = "street.html"

    # Extract images from the Lightroom export file
    images = extract_images_from_lightroom(lightroom_file)

    # Debug: Print all parsed images
    print("Parsed Images:")
    for img in images:
        print(img)

    # Generate HTML content
    html_content = generate_full_html(images, lightroom_folder)

    # Write to the output HTML file
    with open(output_html_file, "w", encoding="utf-8") as file:
        file.write(html_content)

    print(f"HTML page generated: {output_html_file}")

if __name__ == "__main__":
    main()

Parsed Images:
{'id': '18272451', 'largeWidth': 800, 'largeHeight': 1200, 'exportFilename': 'DSC03031', 'title': 'Coffee break (London, UK)'}
{'id': '18541469', 'largeWidth': 1200, 'largeHeight': 800, 'exportFilename': 'DSC04165', 'title': 'Cruising through destruction (Egypt)'}
{'id': '18561870', 'largeWidth': 800, 'largeHeight': 1200, 'exportFilename': 'IMG_0403', 'title': 'Lock em up (Egypt)'}
{'id': '18541708', 'largeWidth': 1200, 'largeHeight': 800, 'exportFilename': 'DSC04404', 'title': 'Open door policy (Egypt)'}
{'id': '18603767', 'largeWidth': 800, 'largeHeight': 1200, 'exportFilename': 'IMG_0936-2', 'title': 'Coffee (Egypt)'}
{'id': '18787653', 'largeWidth': 1200, 'largeHeight': 800, 'exportFilename': 'DSC05868', 'title': 'Walking the highway (Egypt)'}
{'id': '20869300', 'largeWidth': 1200, 'largeHeight': 800, 'exportFilename': 'DSC06845', 'title': 'A lonely guy at a diner (Palo Alto, CA)'}
{'id': '21160647', 'largeWidth': 1200, 'largeHeight': 800, 'exportFilename': 'DSC09218